In [48]:
# Import python libraries
import numpy as np
import pandas as pd
import welleng as we
import plotly.graph_objects as go
from plotly.offline import plot

In [49]:
# List of well file names (adjust as needed)
well_files = ["well1/well1_survey.txt", "well2/well2_survey.txt"]

# Initialize an empty list to store individual well surveys
all_well_surveys = []

# Read data from each well file and create individual surveys
for file_path in well_files:
    well_data = pd.read_csv(file_path, sep="\t")
    columns_to_drop = ["Grav(g)","Mag(Gs)","DipA(deg)"]
    well_data = well_data[well_data['# MD(mKB)'] != '# EOF']
    md_values = well_data['# MD(mKB)'].values
    inc_values = well_data['Incl(deg)'].values
    azi_values = well_data['Azm(deg)'].values
    well_survey = we.survey.Survey(md=md_values, inc=inc_values, azi=azi_values)
    all_well_surveys.append(well_survey)

In [50]:
# Create a 3D scatter plot for each well trajectory
fig = go.Figure()

for i, well_survey in enumerate(all_well_surveys):
    # Interpolate survey data for each well
    well_survey_interp = well_survey.interpolate_survey(step=30)

    # Add interpolated survey data as a line (each well in a different color)
    fig.add_trace(go.Scatter3d(
        x=well_survey_interp.x,
        y=well_survey_interp.y,
        z=well_survey_interp.z,
        mode='lines',
        line=dict(color=f'hsl({i * 360 / len(all_well_surveys)}, 50%, 50%)'),
        name=f'Well {i + 1}'
    ))
    fig.add_trace(go.Scatter3d(
    x=well_survey.x,
    y=well_survey.y,
    z=well_survey.z,
    mode='markers',
    marker=dict(color='red', size=1),  # Adjust marker size here
    name='survey'
))

# Reverse the z-axis for better visualization
fig.update_scenes(zaxis_autorange="reversed")

# Customize the layout
fig.update_layout(
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    scene=dict(
        xaxis_title='East(+)/West(-)',
        yaxis_title='North(+)/South(-)',
        zaxis_title='TVD'
    )
)

# Show the plot
fig.show()